In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("/kaggle/input/shell-25/dataset/train.csv")

In [3]:
df

,Component1_fraction,Component2_fraction,Component3_fraction,Component4_fraction,Component5_fraction,Component1_Property1,Component2_Property1,Component3_Property1,Component4_Property1,Component5_Property1,...,BlendProperty1,BlendProperty2,BlendProperty3,BlendProperty4,BlendProperty5,BlendProperty6,BlendProperty7,BlendProperty8,BlendProperty9,BlendProperty10
0,0.21,0.00,0.42,0.25,0.12,-0.021782,1.981251,0.020036,0.140315,1.032029,...,0.489143,0.607589,0.321670,-1.236055,1.601132,1.384662,0.305850,0.193460,0.580374,-0.762738
1,0.02,0.33,0.19,0.46,0.00,-0.224339,1.148036,-1.107840,0.149533,-0.354000,...,-1.257481,-1.475283,-0.437385,-1.402911,0.147941,-1.143244,-0.439171,-1.379041,-1.280989,-0.503625
2,0.08,0.08,0.18,0.50,0.16,0.457763,0.242591,-0.922492,0.908213,0.972003,...,1.784349,0.450467,0.622687,1.375614,-0.428790,1.161616,0.601289,0.872950,0.660000,2.024576
3,0.25,0.42,0.00,0.07,0.26,-0.577734,-0.930826,0.815284,0.447514,0.455717,...,-0.066422,0.483730,-1.865442,-0.046295,-0.163820,-0.209693,-1.840566,0.300293,-0.351336,-1.551914
4,0.26,0.16,0.08,0.50,0.00,0.120415,0.666268,-0.626934,2.725357,0.392259,...,-0.118913,-1.172398,0.301785,-1.787407,-0.493361,-0.528049,0.286344,-0.265192,0.430513,0.735073
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.50,0.12,0.00,0.26,0.12,0.279523,-0.054170,-0.391227,0.400222,1.032029,...,-0.028366,-0.327297,-0.316933,-1.294092,-0.530259,-0.421526,-0.320869,0.709627,-0.737244,-0.744289
1996,0.19,0.31,0.00,0.37,0.13,-0.887185,0.610050,0.178606,1.083154,-2.822749,...,-0.449245,0.156778,-0.367445,-0.938615,-0.577451,-0.209996,-0.370505,-0.195531,-0.032834,0.269718
1997,0.38,0.06,0.14,0.31,0.11,0.568978,-0.196759,-0.646318,-0.980070,1.032029,...,0.029135,0.164890,-0.092942,-1.134490,-0.437479,-0.695636,-0.101073,0.063650,0.624368,-0.477053
1998,0.50,0.16,0.00,0.18,0.16,-0.067453,0.321977,-0.137535,0.238507,0.017455,...,-0.232960,-0.464947,0.112536,-0.793522,-0.811272,-1.194914,0.100644,0.760116,-0.751394,-0.857598


In [4]:
# !pip install autogluon

In [5]:
import pandas as pd
import numpy as np
from autogluon.tabular import TabularPredictor

# Load data
train_df = pd.read_csv("/kaggle/input/shell-25/dataset/train.csv")
test_df = pd.read_csv("/kaggle/input/shell-25/dataset/test.csv")

# Separate features and targets
feature_cols = train_df.columns[:55]
target_cols = train_df.columns[55:]

X = train_df[feature_cols].copy()
y = train_df[target_cols].copy()

In [6]:
X.columns

Index(['Component1_fraction', 'Component2_fraction', 'Component3_fraction',
       'Component4_fraction', 'Component5_fraction', 'Component1_Property1',
       'Component2_Property1', 'Component3_Property1', 'Component4_Property1',
       'Component5_Property1', 'Component1_Property2', 'Component2_Property2',
       'Component3_Property2', 'Component4_Property2', 'Component5_Property2',
       'Component1_Property3', 'Component2_Property3', 'Component3_Property3',
       'Component4_Property3', 'Component5_Property3', 'Component1_Property4',
       'Component2_Property4', 'Component3_Property4', 'Component4_Property4',
       'Component5_Property4', 'Component1_Property5', 'Component2_Property5',
       'Component3_Property5', 'Component4_Property5', 'Component5_Property5',
       'Component1_Property6', 'Component2_Property6', 'Component3_Property6',
       'Component4_Property6', 'Component5_Property6', 'Component1_Property7',
       'Component2_Property7', 'Component3_Property7', 'C

In [7]:
def compute_volume_weighted_blend_features(X):
    """
    Computes volume-weighted blend-level properties for each of the 10 properties.
    """
    weighted_props = {}
    for prop_idx in range(1, 11):  # Property1 to Property10
        blend_prop = 0
        for comp_idx in range(1, 6):  # Component1 to Component5
            vol_col = f'Component{comp_idx}_fraction'
            prop_col = f'Component{comp_idx}_Property{prop_idx}'
            blend_prop += X[vol_col] * X[prop_col]
        weighted_props[f'BlendEst_Property{prop_idx}'] = blend_prop / 100.0
    return pd.DataFrame(weighted_props)

In [8]:
# Compute and append new blend-level features
blend_features = compute_volume_weighted_blend_features(X)
X = pd.concat([X, blend_features], axis=1)

# Do the same for test data
X_test = test_df[X.columns[:55]]  # assumes same structure
blend_features_test = compute_volume_weighted_blend_features(X_test)
X_test = pd.concat([X_test, blend_features_test], axis=1)

In [9]:
X_test

,Component1_fraction,Component2_fraction,Component3_fraction,Component4_fraction,Component5_fraction,Component1_Property1,Component2_Property1,Component3_Property1,Component4_Property1,Component5_Property1,...,BlendEst_Property1,BlendEst_Property2,BlendEst_Property3,BlendEst_Property4,BlendEst_Property5,BlendEst_Property6,BlendEst_Property7,BlendEst_Property8,BlendEst_Property9,BlendEst_Property10
0,0.18,0.05,0.32,0.37,0.08,-0.177804,-0.741219,0.769821,-0.877069,0.602809,...,-0.000990,0.000756,-0.001855,0.009188,0.004174,0.004803,-0.006113,0.001229,0.000072,-0.000099
1,0.00,0.50,0.00,0.37,0.13,2.501354,0.177344,-0.498739,-0.196742,-1.943463,...,-0.002368,0.001123,0.006133,0.006882,-0.009066,0.004678,0.010493,0.000283,-0.010357,0.004234
2,0.16,0.00,0.17,0.50,0.17,1.547324,0.891479,0.030627,-0.368678,-0.294728,...,0.000183,-0.002463,0.006907,-0.005549,0.010019,0.004685,-0.005192,0.005401,-0.004572,-0.002365
3,0.50,0.00,0.17,0.16,0.17,-0.424427,1.016862,-1.182979,-0.854225,-0.830186,...,-0.006911,0.001502,-0.004316,-0.009140,0.004817,-0.006442,-0.001058,0.014779,-0.005335,-0.001220
4,0.00,0.00,0.50,0.50,0.00,-0.187062,-0.762173,-0.473660,2.074087,0.756849,...,0.008002,-0.014748,0.006398,0.008344,-0.000343,-0.000206,0.000020,-0.005766,-0.001206,-0.001873
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.44,0.01,0.08,0.41,0.06,1.036797,1.415667,0.793302,-0.446630,0.395524,...,0.003744,-0.004471,-0.003875,0.000944,0.006869,-0.003595,0.009048,-0.000716,-0.017260,-0.010985
496,0.19,0.47,0.03,0.23,0.08,-1.305137,-1.520941,-0.989537,0.903203,1.032029,...,-0.007022,0.003248,0.001928,-0.011361,-0.004362,-0.010684,0.006268,0.003925,-0.007330,0.009645
497,0.43,0.01,0.12,0.21,0.23,0.806590,0.607324,0.359058,0.283394,1.032029,...,0.006929,0.013013,-0.001846,0.000177,0.005718,-0.001797,-0.006263,0.003280,-0.009896,0.002967
498,0.03,0.04,0.42,0.42,0.09,-0.792140,0.674275,-1.783487,0.848296,0.164798,...,-0.003747,-0.002723,-0.008590,-0.017343,-0.008161,-0.003805,0.008810,0.000204,-0.002436,0.006742


In [10]:
# After computing blend_features
for i in range(1, 11):
    corr = np.corrcoef(blend_features[f'BlendEst_Property{i}'], train_df[f'BlendProperty{i}'])[0, 1]
    print(f"Corr(Est_Property{i}, True_BlendProperty{i}) = {corr:.3f}")

Corr(Est_Property1, True_BlendProperty1) = 0.475
Corr(Est_Property2, True_BlendProperty2) = 0.483
Corr(Est_Property3, True_BlendProperty3) = -0.029
Corr(Est_Property4, True_BlendProperty4) = 0.510
Corr(Est_Property5, True_BlendProperty5) = 0.250
Corr(Est_Property6, True_BlendProperty6) = 0.678
Corr(Est_Property7, True_BlendProperty7) = 0.372
Corr(Est_Property8, True_BlendProperty8) = 0.507
Corr(Est_Property9, True_BlendProperty9) = 0.446
Corr(Est_Property10, True_BlendProperty10) = 0.311


In [11]:
metric_code = """
import numpy as np
from sklearn.metrics import mean_absolute_percentage_error

def safe_sklearn_mape(y_true, y_pred):
    \"""
    Safe version of Scikit-learn's MAPE that avoids division-by-zero errors
    by replacing small y_true values with epsilon.
    \"""
    epsilon = 1e-6
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_true_safe = np.where(np.abs(y_true) < epsilon, epsilon, y_true)
    return mean_absolute_percentage_error(y_true_safe, y_pred)
"""

# Save to file
with open("mymetrics.py", "w") as f:
    f.write(metric_code.strip())

print("✅ `mymetrics.py` has been created in the current directory.")

✅ `mymetrics.py` has been created in the current directory.


In [12]:
from autogluon.core.metrics import make_scorer
from mymetrics import safe_sklearn_mape

# Register for AutoGluon
mape_metric = make_scorer(
    name='sklearn_safe_mape',
    score_func=safe_sklearn_mape,
    optimum=0,
    greater_is_better=False,
    needs_proba=False,
)

In [ ]:
import os
import shutil
import time
from autogluon.tabular import TabularPredictor
from tqdm import tqdm

model_base_dir = "autogluon_models"
os.makedirs(model_base_dir, exist_ok=True)

predictors = {}
time_limit = 6000  # seconds per model

for target in y.columns:
    model_path = os.path.join(model_base_dir, target)

    # 🧹 Clean up any existing model directory
    if os.path.exists(model_path):
        print(f"\n🧹 Clearing old model for: {target}")
        shutil.rmtree(model_path)

    print(f"\n🚀 Starting training for: {target}")
    start = time.time()

    train_data = pd.concat([X, y[[target]]], axis=1)

    predictor = TabularPredictor(
        label=target,
        path=model_path,
        eval_metric=mape_metric,
        verbosity=2
    ).fit(
        train_data=train_data,
        presets='best_quality',
        time_limit=time_limit,
        num_bag_folds=5,
        num_stack_levels=1,
        # ag_args_fit={"num_gpus": "auto"}
    )

    predictors[target] = predictor

    # Leaderboard
    print(f"\n📊 Leaderboard for {target}")
    print(predictor.leaderboard(silent=True))

    # Training time
    elapsed = time.time() - start
    print(f"✅ Finished training {target} in {elapsed:.2f} sec")

    # Model size
    total_size = sum(os.path.getsize(os.path.join(dp, f)) for dp, _, fns in os.walk(model_path) for f in fns)
    print(f"💾 Model size for {target}: {total_size / 1e6:.2f} MB")


Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Nov 10 10:07:59 UTC 2024
CPU Count:          4
Memory Avail:       29.94 GB / 31.35 GB (95.5%)
Disk Space Avail:   19.50 GB / 19.52 GB (99.9%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=5, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stacke


🧹 Clearing old model for: BlendProperty1

🚀 Starting training for: BlendProperty1


	Running DyStack sub-fit in a ray process to avoid memory leakage. Enabling ray logging (enable_ray_logging=True). Specify `ds_args={'enable_ray_logging': False}` if you experience logging issues.
2025-07-04 19:25:48,256	INFO worker.py:1843 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
		Context path: "/kaggle/working/autogluon_models/BlendProperty1/ds_sub_fit/sub_fit_ho"
(_dystack pid=3195) Running DyStack sub-fit ...
(_dystack pid=3195) Beginning AutoGluon training ... Time limit = 1496s
(_dystack pid=3195) AutoGluon will save models to "/kaggle/working/autogluon_models/BlendProperty1/ds_sub_fit/sub_fit_ho"
(_dystack pid=3195) Train Data Rows:    1777
(_dystack pid=3195) Train Data Columns: 65
(_dystack pid=3195) Label Column:       BlendProperty1
(_dystack pid=3195) Problem Type:       regression
(_dystack pid=3195) Preprocessing data ...
(_dystack pid=3195) Using Feature Generators to preprocess the data ...
(_dystack pid=3195) Fitting AutoMLPipeline

(_ray_fit pid=3404) [1000]	valid_set's l2: 0.0192157	valid_set's sklearn_safe_mape: -0.426266
(_ray_fit pid=3547) [1000]	valid_set's l2: 0.0194699	valid_set's sklearn_safe_mape: -0.378124
(_ray_fit pid=3547) [2000]	valid_set's l2: 0.0194574	valid_set's sklearn_safe_mape: -0.378015


(_dystack pid=3195) 	-0.6407	 = Validation score   (-sklearn_safe_mape)
(_dystack pid=3195) 	23.25s	 = Training   runtime
(_dystack pid=3195) 	0.26s	 = Validation runtime
(_dystack pid=3195) Fitting model: LightGBM_BAG_L1 ... Training model for up to 970.07s of the 1469.02s of remaining time.
(_dystack pid=3195) 	Fitting 5 child models (S1F1 - S1F5) | Fitting with ParallelLocalFoldFittingStrategy (4 workers, per: cpus=1, gpus=0, memory=0.07%)
(_dystack pid=3195) 	-0.8329	 = Validation score   (-sklearn_safe_mape)
(_dystack pid=3195) 	15.55s	 = Training   runtime
(_dystack pid=3195) 	0.04s	 = Validation runtime
(_dystack pid=3195) Fitting model: RandomForestMSE_BAG_L1 ... Training model for up to 951.29s of the 1450.24s of remaining time.
(_dystack pid=3195) /usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes 

...

(_ray_fit pid=5914) 	Ran out of time, early stopping on iteration 8554. Best iteration is: [repeated 2x across cluster]
(_ray_fit pid=5914) 	[8550]	valid_set's l2: 0.0165147	valid_set's sklearn_safe_mape: -0.342092 [repeated 2x across cluster]
(_dystack pid=3195) 	-0.6403	 = Validation score   (-sklearn_safe_mape)
(_dystack pid=3195) 	25.15s	 = Training   runtime
(_dystack pid=3195) 	0.9s	 = Validation runtime
(_dystack pid=3195) Fitting model: NeuralNetTorch_r22_BAG_L1 ... Training model for up to 9.41s of the 508.37s of remaining time.
(_dystack pid=3195) 	Fitting 5 child models (S1F1 - S1F5) | Fitting with ParallelLocalFoldFittingStrategy (4 workers, per: cpus=1, gpus=0, memory=0.02%)
(_ray_fit pid=5959) 	Ran out of time, stopping training early. (Stopping on epoch 13)
(_dystack pid=3195) 	-0.4456	 = Validation score   (-sklearn_safe_mape)
(_dystack pid=3195) 	14.98s	 = Training   runtime
(_dystack pid=3195) 	0.04s	 = Validation runtime
(_ray_fit pid=5956) 	Ran out of time, stopping

(_ray_fit pid=6131) [1000]	valid_set's l2: 0.00632513	valid_set's sklearn_safe_mape: -0.205321 [repeated 2x across cluster]
(_ray_fit pid=6264) [1000]	valid_set's l2: 0.0063929	valid_set's sklearn_safe_mape: -0.349624


(_dystack pid=3195) 	-0.3045	 = Validation score   (-sklearn_safe_mape)
(_dystack pid=3195) 	17.81s	 = Training   runtime
(_dystack pid=3195) 	0.18s	 = Validation runtime
(_dystack pid=3195) Fitting model: LightGBM_BAG_L2 ... Training model for up to 468.26s of the 468.22s of remaining time.
(_dystack pid=3195) 	Fitting 5 child models (S1F1 - S1F5) | Fitting with ParallelLocalFoldFittingStrategy (4 workers, per: cpus=1, gpus=0, memory=0.08%)
(_ray_fit pid=6090) 	Ran out of time, stopping training early. (Stopping on epoch 31)


(_ray_fit pid=6440) [1000]	valid_set's l2: 0.00423885	valid_set's sklearn_safe_mape: -0.206158


(_dystack pid=3195) 	-0.1624	 = Validation score   (-sklearn_safe_mape)
(_dystack pid=3195) 	23.18s	 = Training   runtime
(_dystack pid=3195) 	0.1s	 = Validation runtime
(_dystack pid=3195) Fitting model: RandomForestMSE_BAG_L2 ... Training model for up to 441.76s of the 441.71s of remaining time.
(_dystack pid=3195) /usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
(_dystack pid=3195)   warnings.warn(
(_dystack pid=3195) 	-0.1507	 = Validation score   (-sklearn_safe_mape)
(_dystack pid=3195) 	10.47s	 = Training   runtime
(_dystack pid=3195) 	0.18s	 = Validation runtime
(_dystack pid=3195) Fitting model: CatBoost_BAG_L2 ... Training model for up to 430.99s of the 430.95s of remaining time.
(_dystack pid=3195) 	Fitting 5 child models (S1F1 - S1F5) | Fitti